In [7]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

%matplotlib inline


In [6]:
def readRawImage(url, rows, cols, color=False):
    count = rows*cols*3 if color == True else rows*cols
    shape = (rows, cols, 3) if color == True else (rows, cols)

    fd = open(url, 'rb')
    file = np.fromfile(fd, dtype=np.uint8, count=count)
    fd.close()
    return file.reshape(shape)


def showImage(image, title=""):
    plt.imshow(image, cmap=plt.cm.gray, vmin=0, vmax=255)
    plt.axis("off")
    plt.title(title)


def showImages(datas, cols):
    plt.figure(num=None, figsize=(
        18, 18 * (((len(datas) - 1)//cols) + 1) / cols), dpi=94)
    for index, data in enumerate(datas):
        plt.subplot((len(datas) - 1)//cols + 1, cols, index+1)
        plt.imshow(data["image"], cmap=plt.cm.gray, vmin=0, vmax=255)
        plt.axis("off")
        plt.title(data["title"])


def showImagesHistogram(datas, cols):
    plt.figure(num=None, figsize=(18, 4), dpi=94)
    for index, data in enumerate(datas):
        plt.subplot(len(datas)/cols+1, cols, index+1)
        plt.hist(data["image"].ravel(), bins=256, range=(0, 255))
        plt.title(data["title"])


def showImagesCDF(datas, cols):
    plt.figure(num=None, figsize=(18, 4), dpi=94)
    for index, data in enumerate(datas):
        hist, bin_edges = np.histogram(data["image"], bins=256, range=(0, 255))
        cdf = np.cumsum(hist/256/256)
        plt.subplot(len(datas)/cols+1, cols, index+1)
        plt.plot(cdf)
        plt.title(data["title"] + "\nSuggest threshold: " +
                  str(np.where(cdf > 0.70)[0][0]))


In [3]:
im = cv2.cvtColor(read_raw_image(
    '/usr/src/homeworks/hw4/code/raw/sample1.raw', 256, 256), cv2.COLOR_GRAY2BGR)
cv2.imwrite('output/sample2.jpg', im)


True